# 🚀 YOLOv11 Instance Segmentation Training

Обучение модели **YOLOv11-seg** для сегментации красных кубов и зелёных параллелепипедов.

## Преимущества YOLOv11:
- ⚡ **Быстрее**: Real-time inference (~100+ FPS)
- 🎯 **Точнее**: Улучшенная архитектура vs Mask R-CNN
- 📦 **Проще**: Встроенная аугментация, меньше кода
- 🔧 **Удобнее**: Ultralytics API, автоматический export

## 1. Установка зависимостей

In [ ]:
!pip install ultralytics
!pip install opencv-python-headless
!pip install matplotlib
!pip install tqdm

In [ ]:
import os
import subprocess
import json
from pathlib import Path

from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt

print("✅ Dependencies loaded")

## 2. Загрузка датасета

In [ ]:
# --- ЛОГИКА ЗАГРУЗКИ ДАТАСЕТА ---
REPO_URL = "https://github.com/SergKurchev/strawberry_peduncle_segmentation.git"
REPO_NAME = "strawberry_peduncle_segmentation"
KAGGLE_PATH = "/kaggle/input/strawberry-peduncle-segmentation"

# 1. Проверяем наличие на Kaggle
if os.path.exists(KAGGLE_PATH):
    DATASET_PATH = KAGGLE_PATH
    print(f"✅ Используется датасет из Kaggle Input: {DATASET_PATH}")
else:
    # 2. Клонируем из GitHub
    if not os.path.exists(REPO_NAME):
        print(f"🚀 Клонируем репозиторий из GitHub: {REPO_URL}...")
        subprocess.run(["git", "clone", REPO_URL])
    else:
        print(f"✅ Репозиторий {REPO_NAME} уже существует.")
    
    # Определение пути к dataset
    opt1 = os.path.join(REPO_NAME, "strawberry_peduncle_segmentation", "dataset")
    opt2 = os.path.join(REPO_NAME, "dataset")
    
    if os.path.exists(opt1):
        DATASET_PATH = opt1
    elif os.path.exists(opt2):
        DATASET_PATH = opt2
    else:
        DATASET_PATH = REPO_NAME
        print(f"⚠️ Внимание: Папка 'dataset' не найдена, используем корень репозитория.")

print(f"📍 Итоговый путь к датасету: {DATASET_PATH}")

## 3. Конвертация в YOLO формат

Конвертируем PNG маски в текстовые аннотации с полигонами.

In [ ]:
# Загружаем скрипт конвертации из репозитория
conversion_script = os.path.join(REPO_NAME if not os.path.exists(KAGGLE_PATH) else DATASET_PATH, "convert_to_yolo.py")

if not os.path.exists(conversion_script):
    print("⚠️ Скрипт конвертации не найден. Создаём временный...")
    # Встроенная функция конвертации (упрощённая версия)
    exec(open("convert_to_yolo.py").read())
else:
    print(f"✅ Найден скрипт: {conversion_script}")

# Запускаем конвертацию
!python "{conversion_script}" "{DATASET_PATH}"

YOLO_DATASET_PATH = os.path.join(DATASET_PATH, "yolo")
DATASET_YAML = os.path.join(YOLO_DATASET_PATH, "dataset.yaml")

print(f"\n✅ YOLO dataset ready: {YOLO_DATASET_PATH}")
print(f"   Config: {DATASET_YAML}")

## 4. Обучение YOLOv11

In [ ]:
# Загружаем предобученную модель YOLOv11-seg Large
model = YOLO('yolo11l-seg.pt')

print("✅ YOLOv11-seg Large loaded")
print(f"   Model: yolo11l-seg.pt")
print(f"   Task: Instance Segmentation")

In [ ]:
# Обучение
results = model.train(
    data=DATASET_YAML,
    epochs=50,
    imgsz=1024,
    batch=8,
    device=0,  # GPU
    project='runs/segment',
    name='yolo11_cube_segmentation',
    
    # Аугментации (встроенные в YOLO)
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=15.0,
    translate=0.1,
    scale=0.5,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    
    # Оптимизация
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    
    # Другие параметры
    patience=10,
    save=True,
    save_period=5,
    workers=8,
    verbose=True
)

print("\n✅ Training complete!")

## 5. Валидация

In [ ]:
# Валидация на тестовом сете
metrics = model.val()

print(f"\n📊 Validation Metrics:")
print(f"   mAP50: {metrics.seg.map50:.4f}")
print(f"   mAP50-95: {metrics.seg.map:.4f}")
print(f"   Precision: {metrics.seg.mp:.4f}")
print(f"   Recall: {metrics.seg.mr:.4f}")

## 6. Inference на примерах

In [ ]:
# Загружаем лучшую модель
best_model = YOLO('runs/segment/yolo11_cube_segmentation/weights/best.pt')

# Получаем тестовые изображения
import glob
test_images = glob.glob(os.path.join(DATASET_PATH, "images", "*.png"))[:5]

print(f"Testing on {len(test_images)} images...\n")

for img_path in test_images:
    print(f"\n{'='*80}")
    print(f"📸 {os.path.basename(img_path)}")
    
    # Предсказание
    results = best_model(img_path, conf=0.5)
    
    # Визуализация
    result = results[0]
    
    # Получаем данные
    boxes = result.boxes.xyxy.cpu().numpy()
    classes = result.boxes.cls.cpu().numpy()
    confidences = result.boxes.conf.cpu().numpy()
    
    if result.masks is not None:
        masks = result.masks.data.cpu().numpy()
    else:
        masks = None
    
    print(f"   Detected {len(boxes)} objects:")
    for i, (box, cls, conf) in enumerate(zip(boxes, classes, confidences)):
        class_name = result.names[int(cls)]
        print(f"      [{i}] {class_name}: {conf:.3f}")
    
    # Показываем результат с аннотациями
    annotated = result.plot()
    
    plt.figure(figsize=(15, 10))
    plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    plt.title(f"YOLOv11 Segmentation: {os.path.basename(img_path)}", fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.show()
    
    print(f"{'='*80}")

## 7. Export модели

Экспорт в различные форматы для deployment.

In [ ]:
# Export в ONNX (для Unity, C++, etc.)
onnx_path = best_model.export(format='onnx', simplify=True)
print(f"✅ Exported to ONNX: {onnx_path}")

# Опционально: TorchScript
# torchscript_path = best_model.export(format='torchscript')
# print(f"✅ Exported to TorchScript: {torchscript_path}")

## 📝 Резюме

### Что сделано:
1. ✅ Загрузка датасета (Kaggle / GitHub)
2. ✅ Конвертация PNG masks → YOLO polygons
3. ✅ Обучение YOLOv11-seg Large (50 эпох)
4. ✅ Валидация и метрики
5. ✅ Inference на примерах
6. ✅ Export в ONNX

### Следующие шаги:
- Интегрировать в `depth_estimation_v3_inference.ipynb`
- Заменить Mask R-CNN на YOLOv11
- Сравнить скорость и точность